In [8]:
import requests
import warnings

from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM
warnings.simplefilter("ignore", FutureWarning)

model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True, device_map="cuda").eval()
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True)

# **OCR Benchmark**
***

In [32]:
import random
import os
from tqdm import tqdm
from glob import glob
from PIL import Image, ImageOps

test_set_dir = "/playpen-storage/levlevi/player-re-id/src/testing/ocr_model_comparisons/text_recognition/nba_100_test_set"
image_file_paths = glob(os.path.join(test_set_dir, '*.jpg'))
PROMPT = """Analyze the basketball player shown in the provided still tracklet frame and describe the following details:
1. Jersey Number: Identify the number on the player's jersey. If the player has no jersey, provide None.
Based on the frame description, produce an output prediction in the following JSON format:
{
  "jersey_number": "<predicted_jersey_number>",
}
[EOS]"""

def augment_image(image):
    angle = random.uniform(-1, 1)
    image = image.rotate(angle)
    max_translate = 5
    translate_x = random.uniform(-max_translate, max_translate)
    translate_y = random.uniform(-max_translate, max_translate)
    image = image.transform(
        image.size, Image.AFFINE,
        (1, 0, translate_x, 0, 1, translate_y)
    )
    scale = random.uniform(0.99, 1.01)
    new_size = (int(image.size[0] * scale), int(image.size[1] * scale))
    image = image.resize(new_size, Image.BICUBIC)
    if scale < 1:
        pad_x = (image.size[0] - new_size[0]) // 2
        pad_y = (image.size[1] - new_size[1]) // 2
        image = ImageOps.expand(image, border=(pad_x, pad_y, pad_x, pad_y))
    else:
        crop_x = (new_size[0] - image.size[0]) // 2
        crop_y = (new_size[1] - image.size[1]) // 2
        image = image.crop((crop_x, crop_y, crop_x + image.size[0], crop_y + image.size[1]))
    return image

def is_valid_jersey_number(text):
    if text.isdigit():
        number = int(text)
        return 0 <= number <= 99
    return False

def perform_ocr(image_file_path: str) -> str:
    bootstraped_results = []
    for _ in range(9):
        image = Image.open(image_file_path)
        image = augment_image(image)
        # image = crop_image(image)
        if not image:
            return None
        inputs = processor(text=PROMPT, images=image, return_tensors="pt").to('cuda')
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=1024,
            do_sample=True,
            num_beams=10
        )
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
        parsed_answer = processor.post_process_generation(generated_text, task="<OCR>", image_size=(image.width, image.height))
        bootstraped_results.append(parsed_answer)
    return bootstraped_results

ground_truth_labels = []
results = []
for image_file_path in tqdm(image_file_paths):
    # get human annotation
    ground_truth_label = image_file_path.split('/')[-1].split('_')[1].split('.')[0]
    ground_truth_labels.append(ground_truth_label)
    # perform ocr
    result = perform_ocr(image_file_path)
    results.append(result)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [05:59<00:00,  3.59s/it]


In [40]:
import pandas as pd
import re
from collections import Counter

def get_majority_element(lst):
    counter = Counter(lst)
    return counter.most_common(1)[0][0]

ground_truth_labels = [f.split('/')[-1].split('_')[1].split('.')[0] for f in image_file_paths]
predictions = []
for arr in results:
    pred = None
    raw_results_arr = []
    for r in arr:
        if not r:
            continue
        else:
            pred = r['<OCR>']
        # match only numerical strings
        pred = re.sub('[^0-9]', '', pred)
        if pred == 'unanswerable' or pred == '':
            continue
        raw_results_arr.append(pred)
    # 7+/9 majority vote
    if len(raw_results_arr) > 8:
        pred = get_majority_element(raw_results_arr)
    else:
        pred = 'na'
    predictions.append(pred)
    
df = pd.DataFrame({'ground_truth_label': ground_truth_labels, 'prediction': predictions})
correct = df['ground_truth_label'] == df['prediction']
df['correct'] = correct
df

,ground_truth_label,prediction,correct
0,na,nan,False
1,na,nan,False
2,na,nan,False
3,2,2,True
4,36,nan,False
...,...,...,...
95,41,91,False
96,13,nan,False
97,na,nan,False
98,2,nan,False


In [41]:
tp, fp = [], []
tn, fn = [], []
for row_dict in df.itertuples():
    row = list(row_dict)
    if row[-1] == True:
        tp.append(1)
    elif row[1] == 'na' and row[2] == 'nan':
        tn.append(1)
    elif row[1] == 'na' and row[2] != 'nan':
        fp.append(1)
    else:
        fn.append(1)
        
precision = sum(tp) / (sum(tp) + sum(fp))
recall = sum(tp) / (sum(tp) + sum(fn))

print(sum(tp), '/', sum(tp) + sum(fp))
print(sum(tp), '/', sum(tp) + sum(fn))

19 / 20
19 / 51


In [42]:
19/20

0.95

In [6]:
# precision = tp / (tp + fp)

# **Race Identification Benchmark**
***

In [7]:
import os
from tqdm import tqdm
from glob import glob

test_set_dir = '/mnt/opr/levlevi/player-re-id/src/testing/ocr_model_comparions/nba_100_test_set'
image_file_paths = glob(os.path.join(test_set_dir, '*.jpg'))
PROMPT = """Analyze the basketball player shown in the provided still tracklet frame and describe the following details:
1. Player Race: Identify the players skin color as either 'black', 'white', 'mixed', or 'asian'.
Based on the frame description, produce an output prediction in the following JSON format:
{
  "race": "<predicted_race>",
}
[EOS]"""

def perform_ocr(image_file_path: str) -> str:
    image = Image.open(image_file_path)
    inputs = processor(text=PROMPT, images=image, return_tensors="pt").to('cuda')
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        do_sample=False,
        num_beams=3
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(generated_text, task="<CAPTION_TO_PHRASE_GROUNDING>", image_size=(image.width, image.height))
    return parsed_answer

ground_truth_labels = []
results = []
for image_file_path in tqdm(image_file_paths):
    # get human annotation
    ground_truth_label = image_file_path.split('/')[-1].split('_')[1].split('.')[0]
    ground_truth_labels.append(ground_truth_label)
    # perform ocr
    result = perform_ocr(image_file_path)
    results.append(result)

0it [00:00, ?it/s]
